# Custom Data Configurations <a target="_blank" href="https://colab.research.google.com/github/yWorks/yfiles-jupyter-graphs-for-sparql/blob/main/examples/configurations_example.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install SPARQLWrapper --quiet
%pip install yfiles_jupyter_graphs_for_sparql --quiet
from SPARQLWrapper import SPARQLWrapper, JSON

from yfiles_jupyter_graphs_for_sparql import SparqlGraphWidget

You can also open this notebook in Google Colab when Google Colab's custom widget manager is enabled:

In [ ]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

## Configure data mappings
Configurable values for the subject and object visualization: `text`, `color`, `size`, `styles`, `property`, `coordinate`, `heat` and `type`.

Configurable values for the predicate visualization: `text`, `color`, `thickness_factor`, `heat` and `property`.

When adding a configuration, you always need to specify the type of predicate of the object, subject or predicate you want to adjust. (Here 'member' or 'writer'), additionally you can define the new values for the above mentioned properties. The values may either be:

A lambda the resolves the node or relationship to a specific value.
A constant value which is directly used.
A property-key that points to the value that should be used for the configuration.

In [ ]:

wrapper = SPARQLWrapper("https://dbpedia.org/sparql")
wrapper.setReturnFormat(JSON)
sgw = SparqlGraphWidget(wrapper=wrapper)
query = """SELECT DISTINCT ?sub ?p ?obj
WHERE {
  VALUES ?film {
    <http://dbpedia.org/resource/Pulp_Fiction>
    <http://dbpedia.org/resource/The_Avengers_(2012_film)>
    <http://dbpedia.org/resource/Django_Unchained>
  }
  
  {
    ?film ?p ?obj .
    BIND(?film AS ?sub)
    FILTER(?p IN (dbo:director, dbo:starring, dbo:producer, dbo:cinematography, dbo:musicComposer))
  }
}
LIMIT 40 """

import random
# random heat in between 0.3 and 0.7
def random_heat(item):
    return max(0.3,(min(random.random(), 0.7)))
    

sgw.add_object_configuration('starring', color='purple') #actors
sgw.add_subject_configuration('director', color='blue', heat=random_heat) #movies
sgw.add_predicate_configuration(['producer', 'director'], color='orange', heat=lambda edge: 0.5 if edge['properties']['label'] == 'director' else 0.2) # producer and director edges
sgw.add_predicate_configuration('*', thickness_factor= 2) # all edges, where no specific configuration is given
sgw.show_query(query)

In [ ]:
sgw.add_parent_configuration('starring', True)
sgw.add_parent_configuration('director')
sgw.show_query(query, layout='orthogonal')

### Delete a configuration

Configurations are stored within the class instance and must be explicitly removed to no longer be applied.

In [ ]:
sgw.del_subject_configuration('director')
sgw.show_query(query, layout='orthogonal')